In [1]:
import torch
import math
import numpy as np
import matplotlib.pyplot as plt
import random
%matplotlib inline

In [3]:
class Neuron:
    
    def __init__(self, nin): # nin is the number of inputs to the neuron
        self.w = (torch.rand(nin) * 2) - 1
        self.w.requires_grad = True        
        self.b = (torch.rand(1) * 2) - 1
        self.b.requires_grad = True
        
    def __call__(self, x):
        act = sum((wi*xi for wi, xi in (zip(self.w, x))), self.b) + self.b
        out = act.tanh()
        return out
    
    def parameters(self):
        return [self.w, self.b]

class Layer:
    
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]
        
    def __call__(self, x):
        out = [n(x) for n in self.neurons]
        return out[0] if len(out) == 1 else out

    def parameters(self):
        params = []
        for neuron in self.neurons:
            ps = neuron.parameters()
            params.extend(ps)
        return params
    
class MLP:
    
    def __init__(self, nin, nout):
        sz = [nin] + nout
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nout))]
        
    def __call__(self, x):        
        for layer in self.layers:
            x = layer(x)
        return x
    
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

In [109]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0]
]
ys = [1.0, -1.0, -1.0, 1.0]

xs = torch.tensor(xs) ; xs.requires_grad = True
ys = torch.tensor(ys) ; ys.requires_grad = True

n = MLP(3, [4, 4, 1])

In [418]:
# forward pass

ypred = [n(x) for x in xs]
loss = sum((ygrt - yout)**2 for ygrt, yout in zip(ys, ypred))
loss

tensor([0.0002], grad_fn=<AddBackward0>)

In [419]:
# backward pass

loss.backward()

In [420]:
# update

for p in n.parameters():
    p.data += -0.001 * p.grad

In [421]:
ypred

[tensor([0.9928], grad_fn=<TanhBackward0>),
 tensor([-0.9922], grad_fn=<TanhBackward0>),
 tensor([-0.9922], grad_fn=<TanhBackward0>),
 tensor([0.9946], grad_fn=<TanhBackward0>)]